In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
import time

In [2]:
start = time.time()
from minhash import MinHashGenerator

generator = MinHashGenerator(
    seed = 42,
    num_hashes = 128,
    char_ngrams = 4,
    column_field = "code"
)
input_path = "/content/train-00000-of-00001.parquet"
output_path_minhash = "/content/output_minhash.parquet"
df_ = generator(input_path, output_path_minhash)

Loading data from /content/train-00000-of-00001.parquet...
Computing MinHash signatures...
Saving results to /content/output_minhash.parquet...
MinHash signatures saved to /content/output_minhash.parquet


In [3]:
# import pandas as pd

# df = pd.read_parquet(output_path_minhash)
# print(df.shape)
# df.head()

In [4]:
# df = pd.read_parquet(input_path)
# df.head()

In [5]:
# df_

In [6]:
from lsh import SimpleLSH

l = SimpleLSH(
    num_hashes = 128,
    num_buckets = 16,
    id_field = "id"
)
output_path_lsh = "/content/output_lsh.parquet"
df__ = l(output_path_minhash, output_path_lsh)

/content/lsh.py:135: UserWarning: Output path /content/output_lsh.parquet already exists and will be overwritten
  self._write_buckets(df_batch, output_parquet, overwrite=not wrote_buckets)


In [7]:
# df = pd.read_parquet(output_path_lsh)
# print(df.shape)
# df.head()

In [8]:
# df__

In [9]:
# grouped_docs = df.groupby('_bucket_id')['id'].apply(list).reset_index()
# grouped_docs.columns = ['bucket_id', 'document_ids']
# print(grouped_docs.shape)
# print(grouped_docs.head())

In [10]:
# bucket_sizes = grouped_docs['document_ids'].apply(len)
# print(bucket_sizes.value_counts().sort_index())

In [11]:
# grouped_docs.tail()

In [12]:
from buckets_edges import BucketsToEdges

edges = BucketsToEdges(bucket_field="_bucket_id")
output_path_edges = "/content/output_edges.parquet"
df_edges = edges(output_path_lsh, output_path_edges)

Starting conversion of LSH Buckets to Graph Edgelist
Reading input from: /content/output_lsh.parquet
Time taken for Converted Buckets To Edgelist = 0.14931106567382812s and output written at /content/output_edges.parquet


<ipython-input-12-ad163f25a953>:5: UserWarning: Output path /content/output_edges.parquet already exists and will be overwritten
  df_edges = edges(output_path_lsh, output_path_edges)


In [13]:
# df = pd.read_parquet(output_path_edges)
# print(df.shape)
# df.head()

In [14]:
# df.tail()

In [15]:
# from connectedcomponents import ConnectedComponents

# cc = ConnectedComponents(
#     left_id_col="id_x",
#     right_id_col="id_y"
# )
# output_path_cc = "/content/output_cc.parquet"
# result_df = cc(
#     output_path_edges,
#     output_path_cc
# )

In [16]:
from connectedcomponents import simple_connected_components

output_path_cc = "/content/output_cc.parquet"
result_df = simple_connected_components(
    edges_path=output_path_edges,
    left_id_col="id_x",
    right_id_col="id_y",
    jaccard_col="jaccard",
    jaccard_threshold=0.8,
    output_path=output_path_cc
)

print(f"Found {result_df['group_id'].nunique()} duplicate groups")

Loading edges...
Loaded 759 edges
Columns: ['id_x', 'id_y', 'jaccard']
Filtering by jaccard >= 0.8
After filtering: 759 edges
Creating graph...
Running connected components...
Found 85 connected components
Result shape: (348, 2)
Saving to /content/output_cc.parquet
Found 85 duplicate groups


In [17]:
import pandas as pd
df=pd.read_parquet(output_path_cc)
# print(df.shape)
# df.head()

In [18]:
# df.tail()

In [19]:
# result

In [20]:
# # Check group sizes
# group_sizes = result_df.groupby('group_id').size().sort_values(ascending=False)
# print("Group size distribution:")
# print(group_sizes.value_counts().sort_index())

In [21]:
# Assuming result_df has columns: 'node_id', 'group_id'
grouped_df = df.groupby("group_id")["node_id"].apply(list).reset_index()
grouped_df

,group_id,node_id
0,30,"[4707, 2547, 1967, 4094, 4565, 4563, 4861, 111..."
1,49,"[2756, 3423, 2204, 1003, 1693, 1974, 2139, 367..."
2,77,"[1323, 2304, 3997, 467, 1341, 3327, 3358, 3972..."
3,104,"[2177, 490, 611, 2375]"
4,107,"[360, 1246, 2251, 3458]"
...,...,...
80,341,"[1422, 1423, 2813, 3967, 751, 752, 4831]"
81,343,"[3396, 931, 4851]"
82,344,"[2924, 4862]"
83,345,"[703, 1618, 4125, 2282, 3213, 626, 4885]"


In [22]:
total_time = time.time()-start
print(total_time)

9.240106582641602
